In [1]:
import os
import sys
import scipy.io
import imageio
import scipy.misc
from scipy import io
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import pprint                  #to format printing of the vgg model
%matplotlib inline
import tensorflow.compat.v1 as tf
from utils import *

In [5]:
def total_style_cost(STYLE_LAYERS,model):
    J_style_total=0
    for layer,coeff in STYLE_LAYERS:
        a_S=sess.run(model[layer])
        a_G=model[layer]
        layer_cost=layer_style_cost(a_S,a_G)
        J_style_total+=coeff*layer_cost
    return J_style_total

In [123]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
sess=tf.compat.v1.InteractiveSession()

In [124]:
def load_vgg_model(path):
    vgg = io.loadmat(path)
    vgg_layers = vgg['layers']
    def _weights(layer, expected_layer_name):
        wb = vgg_layers[0][layer][0][0][2]
        W = wb[0][0]
        b = wb[0][1]
        layer_name = vgg_layers[0][layer][0][0][0][0]
        assert layer_name == expected_layer_name
        return W, b
        return W, b
    def _relu(conv2d_layer):
        return tf.nn.relu(conv2d_layer)
    def _conv2d(prev_layer, layer, layer_name):    
        W, b = _weights(layer, layer_name)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(prev_layer, filters=W, strides=[1, 1, 1, 1], padding='SAME') + b
    def _conv2d_relu(prev_layer, layer, layer_name):
        return _relu(_conv2d(prev_layer, layer, layer_name))
    
    def _avgpool(prev_layer):
        return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Constructs the graph model.
    graph = {}
    graph['input']   = tf.Variable(np.zeros((1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)), dtype = 'float32')
    graph['conv1_1']  = _conv2d_relu(graph['input'], 0, 'conv1_1')
    graph['conv1_2']  = _conv2d_relu(graph['conv1_1'], 2, 'conv1_2')
    graph['avgpool1'] = _avgpool(graph['conv1_2'])
    graph['conv2_1']  = _conv2d_relu(graph['avgpool1'], 5, 'conv2_1')
    graph['conv2_2']  = _conv2d_relu(graph['conv2_1'], 7, 'conv2_2')
    graph['avgpool2'] = _avgpool(graph['conv2_2'])
    graph['conv3_1']  = _conv2d_relu(graph['avgpool2'], 10, 'conv3_1')
    graph['conv3_2']  = _conv2d_relu(graph['conv3_1'], 12, 'conv3_2')
    graph['conv3_3']  = _conv2d_relu(graph['conv3_2'], 14, 'conv3_3')
    graph['conv3_4']  = _conv2d_relu(graph['conv3_3'], 16, 'conv3_4')
    graph['avgpool3'] = _avgpool(graph['conv3_4'])
    graph['conv4_1']  = _conv2d_relu(graph['avgpool3'], 19, 'conv4_1')
    graph['conv4_2']  = _conv2d_relu(graph['conv4_1'], 21, 'conv4_2')
    graph['conv4_3']  = _conv2d_relu(graph['conv4_2'], 23, 'conv4_3')
    graph['conv4_4']  = _conv2d_relu(graph['conv4_3'], 25, 'conv4_4')
    graph['avgpool4'] = _avgpool(graph['conv4_4'])
    graph['conv5_1']  = _conv2d_relu(graph['avgpool4'], 28, 'conv5_1')
    graph['conv5_2']  = _conv2d_relu(graph['conv5_1'], 30, 'conv5_2')
    graph['conv5_3']  = _conv2d_relu(graph['conv5_2'], 32, 'conv5_3')
    graph['conv5_4']  = _conv2d_relu(graph['conv5_3'], 34, 'conv5_4')
    graph['avgpool5'] = _avgpool(graph['conv5_4'])
    
    return graph

In [125]:
model=load_vgg_model('model_pretrained/imagenet-vgg-verydeep-19.mat')

In [133]:
content_image=imageio.imread('images/camp-nou.jpg')
content_image=resize_and_normalize(content_image)

C:\Users\kary1\AppData\Local\Temp\ipykernel_31164\577542952.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  content_image=imageio.imread('images/camp-nou.jpg')


In [134]:
sess.run(model['input'].assign(content_image))
a_C=sess.run(model['conv4_2'])
a_G=model['conv4_2']
cont_cost=content_cost(a_C,a_G)

In [135]:
style_image=imageio.imread('images/drop-of-water.jpg')
style_image=resize_and_normalize(style_image)

C:\Users\kary1\AppData\Local\Temp\ipykernel_31164\2597654123.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  style_image=imageio.imread('images/drop-of-water.jpg')


In [136]:
sess.run(model['input'].assign(style_image))
style_cost=total_style_cost(STYLE_LAYERS,model)

In [137]:
alpha=40
beta=10

In [138]:
J=total_cost(cont_cost,style_cost,alpha,beta)

In [139]:
optimizer=tf.compat.v1.train.AdamOptimizer(2)
train_step=optimizer.minimize(J)

In [140]:
cont=imageio.imread('images/camp-nou.jpg')
generated_image=generate_noise_image(cont,noise_ratio=.5)
generated_image=resize_and_normalize(generated_image)

C:\Users\kary1\AppData\Local\Temp\ipykernel_31164\3295251339.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  cont=imageio.imread('images/camp-nou.jpg')


In [141]:
def train(sess,input_image,num_iters=1000):
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(model['input'].assign(input_image))
    generated_im=input_image
    for i in range(num_iters):
        sess.run(train_step)
        generated_im=sess.run(model['input'])
        if i%100==0:
            Jt, Jc, Js = sess.run([J, cont_cost, style_cost])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))

            save_image('generated/'+str(i)+'.png',generated_im)
    save_image('generated/final_image.png',generated_im)

In [142]:
train(sess,generated_image)

Iteration 0 :
total cost = 8.777952e+21
content cost = 13645177000.0
style cost = 8.777952e+20
Iteration 100 :
total cost = 4.9697534e+19
content cost = 40243413000.0
style cost = 4.9697535e+18
Iteration 200 :
total cost = 1.8949487e+19
content cost = 44396302000.0
style cost = 1.8949485e+18
Iteration 300 :
total cost = 1.3848478e+19
content cost = 46152315000.0
style cost = 1.3848475e+18
Iteration 400 :
total cost = 1.1136918e+19
content cost = 47292207000.0
style cost = 1.1136916e+18
Iteration 500 :
total cost = 9.266377e+18
content cost = 48208933000.0
style cost = 9.266375e+17
Iteration 600 :
total cost = 7.853351e+18
content cost = 48989532000.0
style cost = 7.8533484e+17
Iteration 700 :
total cost = 6.752369e+18
content cost = 49706000000.0
style cost = 6.752367e+17
Iteration 800 :
total cost = 5.8805576e+18
content cost = 50377000000.0
style cost = 5.880555e+17
Iteration 900 :
total cost = 5.183232e+18
content cost = 51036676000.0
style cost = 5.1832298e+17
